In [1]:
import webotsgym as wg

from webotsgym.config import WebotConfig
from webotsgym.environment import webotsgym
from webotsgym.evaluate import Evaluate, EvaluateMats, EvaluatePJ0
from webotsgym.action import DiscreteAction, ContinuousAction
from webotsgym.observation import Observation


import numpy as np
import james

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset
from stable_baselines.gail import generate_expert_traj

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
class MyObs(Observation):
    def __init__(self, env):
        super(MyObs, self).__init__(env)
        self.env = env

class MyEval(Evaluate):
    def __init__(self, env, config: WebotConfig = WebotConfig()):
        super(MyEval, self).__init__(env, config)
        self.reward_range = (-100, 100)

    def calc_reward(self):
        if self.env.get_target_distance() < 0.1:
            return self.reward_range[1]
        else:
            dist = self.env.get_target_distance()
            denom = self.env.max_distance / 2
            return -1 * np.tanh(dist / denom)

    def check_done(self):
        if self.env.iterations % self.config.reset_env_after == 0:
            return True
        if self.env.get_target_distance() < 0.25:
            return True
        return False
    

config = WebotConfig()
config.fast_simulation = False
config.reset_env_after = 20000
config.num_obstacles = 0
config.world_size = 4
action_class = ContinuousAction(direction_type="steering", relative=False)

env = webotsgym(train=True, 
                action_class=action_class, 
                evaluate_class=MyEval,
                observation_class=MyObs,
                config=config)

/home/ruili/anaconda3/envs/spinningup/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Accepting on Port:  10201
sending: env


In [3]:

def cb(action):
    
    s,r,d,i = env.step((action.heading, action.speed))
    if d is True:
        return

k = james.Keyboard(cb)

def human_expert(_obs):

    k.action()
    return (k.act.heading, k.act.speed)

n_epi = 1  #number of expert trajectories
expert_name = "expert_human_ppo1_webots_continues_{}".format(n_epi)
generate_expert_traj(human_expert, expert_name, env, n_episodes=n_epi)

#traj_limitation is the number of expert trajectories that you want to use for pretraining
dataset = ExpertDataset(expert_path="{}.npz".format(expert_name), traj_limitation=1, batch_size=128)
env.reset()
model = PPO1("MlpPolicy", env, verbose=1)
model.pretrain(dataset, n_epochs=1000)

sending: env
1
1
sending: env
actions (2, 2)
obs (2, 368)
rewards (2,)
episode_returns (1,)
episode_starts (2,)
actions (2, 2)
obs (2, 368)
rewards (2,)
episode_returns (1,)
episode_starts (2,)
Total trajectories: 1
Total transitions: 2
Average returns: -0.19846181398716342
Std for returns: 0.0
sending: env




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Pretraining with Behavior Cloning...
==== Training progress 10.00% ====
Epoch 100
Training loss: 0.000000, Validation loss: 0.003964

==== Training progress 20.00% ====
Epoch 200
Training loss: 0.000000, Validation loss: 0.003967

==== Training progress 30.00% ====
Epoch 300
Training loss: 0.000000, Validation loss: 0.003966

==== Training progress 40.00% ====
Epoch 400
Training loss: 0.000000, Validation loss: 0.003966

==== Training progress 50